In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
LOG_PATH="/Users/law/repos/ma/disco/benchmark-runs/runs_sustainable_throughput"

In [24]:
import os
import re

RUN_RE = re.compile(r"Running (\d+) child.*, (\d+) stream.*")
THROUGHPUT_RE = re.compile(r"Found sustainable candidate \((\d+) events/s.\)*")

def parse_log_file(log_file):
    sustainable_throughputs= {}
    
    current_run = None
    current_throughput = None
    with open(log_file) as f:
        for line in f:
            run_match = RUN_RE.match(line)
            if run_match is not None:
                if current_run != None:
                    print(f"Did not find candidate line for {current_run}")
                current_run = (int(run_match.group(1)), int(run_match.group(2)))
                current_throughput = None

            throughput_match = THROUGHPUT_RE.match(line)
            if throughput_match is not None:
                if current_throughput is not None:
                    print(f"Did not find run line after {current_run}")
                current_throughput = int(throughput_match.group(1))
                sustainable_throughputs[current_run] = current_throughput
                current_run = None
                
    return sustainable_throughputs
                

ALL_THROUGHPUTS = {}
for log_file in sorted(os.listdir(LOG_PATH)):
    if log_file.endswith("-nodes-max.log"):
        print(f"Parsing {log_file}")
        sustainable_throughputs = parse_log_file(os.path.join(LOG_PATH, log_file))
        print(f"current: {sustainable_throughputs}")
        ALL_THROUGHPUTS = {**ALL_THROUGHPUTS, **sustainable_throughputs}
        print(f"all:     {ALL_THROUGHPUTS}\n")        

Parsing sustainable_run_2019-27-08-1116-8-nodes-max.log
Did not find candidate line for (1, 2)
current: {(1, 1): 644531, (1, 4): 240234, (1, 8): 117187}
all:     {(1, 1): 644531, (1, 4): 240234, (1, 8): 117187}

Parsing sustainable_run_2019-27-08-1748-1-child-8-nodes-max.log
current: {(1, 1): 650390, (1, 2): 445312, (1, 4): 222656, (1, 8): 117187}
all:     {(1, 1): 650390, (1, 4): 222656, (1, 8): 117187, (1, 2): 445312}

Parsing sustainable_run_2019-27-08-1901-1-child-1-nodes-max.log
current: {(1, 1): 906250}
all:     {(1, 1): 906250, (1, 4): 222656, (1, 8): 117187, (1, 2): 445312}



In [25]:
for (num_children, num_streams), throughput in sorted(ALL_THROUGHPUTS.items()):
    print(f"Sustainable throughput for {num_children} child(ren) with " \
          f"{num_streams // num_children} stream(s) each " \
          f"is {throughput * num_streams} events/s.")

Sustainable throughput for 1 child(ren) with 1 stream(s) each is 906250 events/s.
Sustainable throughput for 1 child(ren) with 2 stream(s) each is 890624 events/s.
Sustainable throughput for 1 child(ren) with 4 stream(s) each is 890624 events/s.
Sustainable throughput for 1 child(ren) with 8 stream(s) each is 937496 events/s.


# Plots

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True, 'pgf.rcfonts' : False})